### Mudança de preço dos combustíveis automotivos ao longo do tempo (2004 a 2024)

**Autor:** Antonino Marques Jares

**Fonte:** https://dados.gov.br/dados/conjuntos-dados/serie-historica-de-precos-de-combustiveis-e-de-glp

**Atualizado em:** 04/06/2025

# Passo 1:
Crie as pastas csv e csv_resumo

# Passo 2:
Baixe os dataset's em https://dados.gov.br/dados/conjuntos-dados/serie-historica-de-precos-de-combustiveis-e-de-glp

Todos 1º e 2º Semestres dos anos 2004 a 2024 (total de 42 arquivos csv) e salvando todos na pasta csv.

# Criando primeiro resumos na pasta csv_resumo

In [2]:
import pandas as pd
from pathlib import Path
import warnings

# Suprime avisos de dtype (opcional, mas recomendado para evitar poluição no output)
warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)

# Lista de anos (2004 a 2024)
anos = list(range(2004, 2025))

for ano in anos:
    for semestre in ['01', '02']:
        try:
            # Define caminhos dos arquivos
            planilha_origem = Path(f"C:/Users/Nino/AnacondaProjects/combustivel/csv/ca-{ano}-{semestre}.csv")
            planilha_destino = Path(f"C:/Users/Nino/AnacondaProjects/combustivel/csv_resumo/ca-{ano}-{semestre}.csv")
            
            # Verifica se o arquivo existe
            if not planilha_origem.exists():
                print(f"⚠ Arquivo não encontrado: {planilha_origem}")
                continue  # Pula para o próximo arquivo

            # Tenta ler com diferentes codificações e ignora dtype misto
            try:
                df = pd.read_csv(planilha_origem, sep=";", encoding='utf-8', low_memory=False)
            except UnicodeDecodeError:
                try:
                    df = pd.read_csv(planilha_origem, sep=";", encoding='latin1', low_memory=False)
                except UnicodeDecodeError:
                    df = pd.read_csv(planilha_origem, sep=";", encoding='cp1252', low_memory=False)

            # Verifica se as colunas necessárias existem
            colunas_necessarias = [
                'Regiao - Sigla', 'Estado - Sigla', 'Municipio',
                'CNPJ da Revenda', 'Produto', 'Data da Coleta', 'Valor de Venda'
            ]
            
            colunas_faltantes = [col for col in colunas_necessarias if col not in df.columns]
            if colunas_faltantes:
                print(f"⚠ Colunas faltantes em {planilha_origem}: {colunas_faltantes}")
                continue  # Pula se não tiver todas as colunas

            # Seleciona e renomeia colunas
            df = df[colunas_necessarias].rename(columns={
                'Estado - Sigla': 'ESTADO',
                'Municipio': 'MUNICIPIO',
                'CNPJ da Revenda': 'CNPJ',
                'Produto': 'PRODUTO',
                'Data da Coleta': 'DATA',
                'Valor de Venda': 'VL_VENDA'
            })

            # Processa datas (converte e extrai ano/mês)
            df['DATA'] = pd.to_datetime(df['DATA'], errors='coerce')
            df['ANO'] = df['DATA'].dt.year.astype('Int64')
            df['MES'] = df['DATA'].dt.month.astype('Int64')
            # Formato 01/04 em vez de Jan/04
            df['MES_ANO'] = df['MES'].astype(str).str.zfill(2) + '/' + df['ANO'].astype(str)

            # Salva o arquivo processado
            df.to_csv(planilha_destino, index=False, encoding='utf-8')
            print(f"✅ {planilha_destino} salvo com sucesso!")

        except Exception as e:
            print(f"❌ Erro grave em {planilha_origem}: {str(e)}")

print("✅ Todos os arquivos processados!")

✅ C:\Users\Nino\AnacondaProjects\combustivel\csv_resumo\ca-2004-01.csv salvo com sucesso!
✅ C:\Users\Nino\AnacondaProjects\combustivel\csv_resumo\ca-2004-02.csv salvo com sucesso!
✅ C:\Users\Nino\AnacondaProjects\combustivel\csv_resumo\ca-2005-01.csv salvo com sucesso!
✅ C:\Users\Nino\AnacondaProjects\combustivel\csv_resumo\ca-2005-02.csv salvo com sucesso!
✅ C:\Users\Nino\AnacondaProjects\combustivel\csv_resumo\ca-2006-01.csv salvo com sucesso!
✅ C:\Users\Nino\AnacondaProjects\combustivel\csv_resumo\ca-2006-02.csv salvo com sucesso!
✅ C:\Users\Nino\AnacondaProjects\combustivel\csv_resumo\ca-2007-01.csv salvo com sucesso!
✅ C:\Users\Nino\AnacondaProjects\combustivel\csv_resumo\ca-2007-02.csv salvo com sucesso!
✅ C:\Users\Nino\AnacondaProjects\combustivel\csv_resumo\ca-2008-01.csv salvo com sucesso!
✅ C:\Users\Nino\AnacondaProjects\combustivel\csv_resumo\ca-2008-02.csv salvo com sucesso!
✅ C:\Users\Nino\AnacondaProjects\combustivel\csv_resumo\ca-2009-01.csv salvo com sucesso!
✅ C:\Users

# Verificando se todas as colunas desejadas existem no DataFrame

In [3]:
colunas_selecionadas = ['ESTADO', 'PRODUTO', 'MES_ANO', 'VL_VENDA']
colunas_disponiveis = df.columns.tolist()
colunas_faltantes = [col for col in colunas_selecionadas if col not in colunas_disponiveis]

if colunas_faltantes:
    print(f"\nAtenção: Colunas não encontradas - {colunas_faltantes}")
else:
    df_filtrado = df[colunas_selecionadas].copy()
    print(df_filtrado.head(10))

  ESTADO             PRODUTO  MES_ANO VL_VENDA
0     AC            GASOLINA  01/2024     6,79
1     AC  GASOLINA ADITIVADA  01/2024     6,84
2     AC              DIESEL  01/2024     6,99
3     AC          DIESEL S10  01/2024     6,99
4     AC              ETANOL  01/2024     4,69
5     AC            GASOLINA  01/2024     6,79
6     AC  GASOLINA ADITIVADA  01/2024     6,89
7     AC              DIESEL  01/2024     7,04
8     AC          DIESEL S10  01/2024     7,09
9     AC              ETANOL  01/2024     4,69


# Criando tabela combustivel no SQLITE

In [4]:
import pandas as pd
import sqlite3
from pathlib import Path

def drop_tabela():
    try:
        cursor.execute("""
        DROP TABLE combustivel
        """)
        conn.commit()
    except FileNotFoundError:
        print(f"Tabela combustivel não foi apagada")
    except Exception as e:
        print(f"Erro ao processar {ano}: {str(e)}")
    finally:
        print("fim drop_tabela")

def criar_tabela():
    try:
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS combustivel (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            ESTADO TEXT,
            PRODUTO TEXT,
            MES_ANO TEXT,
            VL_VENDA FLOAT
        )
        """)
        conn.commit()
    except FileNotFoundError:
        print(f"Tabela combustivel não criada")
    except Exception as e:
        print(f"Erro ao processar {ano}: {str(e)}")
    finally:
        print("fim drop_tabela")


try:

    # Conectar ao banco de dados SQLite
    db_path = r"C:\Users\Nino\AnacondaProjects\combustivel\combustivel.db"
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Apaga a tabela se existir
    drop_tabela()

    # Criar a tabela se não existir
    criar_tabela()

    # Verificar se a tabela existe e suas colunas
    cursor = conn.cursor()
    cursor.execute("PRAGMA table_info(combustivel)")
    
    # Carregar os arquivos CSV
    anos = list(range(2004, 2025))
    for ano in anos:
        for semestre in ['01','02']:
            print(f"\nProcessando ano ({ano}) / semestre ({semestre}) ...")
            try:
                arquivo = Path(rf"C:\Users\Nino\AnacondaProjects\combustivel\csv_resumo\ca-{ano}-{semestre}.csv")
                df = pd.read_csv(arquivo, sep=',', index_col=0, encoding='latin1', low_memory=False)
                # Selecionar apenas as colunas desejadas
                df_filtrado = df[['ESTADO', 'PRODUTO', 'MES_ANO', 'VL_VENDA']].copy()

                # Salvar o DataFrame na tabela SQLite
                df_filtrado.to_sql(
                    name='combustivel',         # Nome da tabela
                    con=conn,                   # Conexão com o banco
                    if_exists='append',         # Adiciona os dados à tabela existente
                    index=False                 # Não inclui o índice do DataFrame
                )

                print(f"Dados de {ano} salvos. \nRegistros salvos: {len(df_filtrado)}")
            
            except FileNotFoundError:
                print(f"Arquivos do ano {ano} não encontrados. Pulando...")
            except Exception as e:
                print(f"Erro ao processar {ano}: {str(e)}")

    print("\nProcesso concluído com sucesso!")
    conn.close()
except Exception as e:
    print(f"\nERRO CRÍTICO: {str(e)}")

finally:
    if 'conn' in locals():
        conn.close()

fim drop_tabela
fim drop_tabela

Processando ano (2004) / semestre (01) ...
Dados de 2004 salvos. 
Registros salvos: 281531

Processando ano (2004) / semestre (02) ...
Dados de 2004 salvos. 
Registros salvos: 915317

Processando ano (2005) / semestre (01) ...
Dados de 2005 salvos. 
Registros salvos: 912729

Processando ano (2005) / semestre (02) ...
Dados de 2005 salvos. 
Registros salvos: 741813

Processando ano (2006) / semestre (01) ...
Dados de 2006 salvos. 
Registros salvos: 868880

Processando ano (2006) / semestre (02) ...
Dados de 2006 salvos. 
Registros salvos: 802334

Processando ano (2007) / semestre (01) ...
Dados de 2007 salvos. 
Registros salvos: 843679

Processando ano (2007) / semestre (02) ...
Dados de 2007 salvos. 
Registros salvos: 659632

Processando ano (2008) / semestre (01) ...
Dados de 2008 salvos. 
Registros salvos: 636862

Processando ano (2008) / semestre (02) ...
Dados de 2008 salvos. 
Registros salvos: 678557

Processando ano (2009) / semestre (01) ...
Dado

# Calculando o preço médio dos combustíveis e salvando em preco_medio_combustiveis.csv

In [5]:
import pandas as pd
import sqlite3

try:
    # Conectar ao banco de dados
    caminho_sqlite = rf"C:\Users\Nino\AnacondaProjects\combustivel\combustivel.db"
    conn = sqlite3.connect(caminho_sqlite)

    query = """
    SELECT 
        ESTADO,
        PRODUTO,
        MES_ANO,
        AVG(VL_VENDA) AS VL_VENDA
    FROM 
        combustivel
    WHERE 
        MES_ANO IS NOT NULL 
        AND VL_VENDA IS NOT NULL
        AND VL_VENDA > 0
    GROUP BY 
        ESTADO, PRODUTO, MES_ANO
    ORDER BY 
        ESTADO, PRODUTO, MES_ANO; 
    """  
    
    # Carregar os dados
    df = pd.read_sql_query(query, conn)
    
    planilha_resumo = rf"C:\Users\Nino\AnacondaProjects\combustivel\preco_medio_combustiveis.csv"
    print(f"Sanvando ... preco_medio_combustiveis.csv")
    df.to_csv(planilha_resumo)
    
    conn.close()
except Exception as e:
    print(f"Erro ao acessar o banco de dados: {e}")

finally:
    # Garante que a conexão será fechada mesmo se ocorrer erro
    if 'conn' in locals():
        conn.close()
    print("\nConexão com o banco de dados encerrada.")

Sanvando ... preco_medio_combustiveis.csv

Conexão com o banco de dados encerrada.


# Separando a informação por tipo de combustível 

In [6]:
import pandas as pd

try:
    # Conectar ao banco de dados
    planilha = rf"C:\Users\Nino\AnacondaProjects\combustivel\preco_medio_combustiveis.csv"
    df = pd.read_csv(planilha, sep=',', index_col=0, encoding='latin1', low_memory=False)
    
    # Filtrar apenas GASOLINA
    df_gasolina = df[df['PRODUTO'] == 'GASOLINA'].copy()
    resumo_gasolina = rf"C:\Users\Nino\AnacondaProjects\combustivel\resumo_gasolina.csv"
    print(f"Sanvando ... resumo_gasolina.csv")
    df_gasolina.to_csv(resumo_gasolina)

    # Filtrar apenas ETANOL
    df_etanol = df[df['PRODUTO'] == 'ETANOL'].copy()
    resumo_etanol = rf"C:\Users\Nino\AnacondaProjects\combustivel\resumo_etanol.csv"
    print(f"Sanvando ... resumo_etanol.csv")
    df_etanol.to_csv(resumo_etanol)

    # Filtrar apenas DIESEL
    df_diesel = df[df['PRODUTO'] == 'DIESEL'].copy()
    resumo_diesel = rf"C:\Users\Nino\AnacondaProjects\combustivel\resumo_diesel.csv"
    print(f"Sanvando ... resumo_diesel.csv")
    df_diesel.to_csv(resumo_diesel)
    
except Exception as e:
    print(f"Erro ao acessar o csv: {e}")

finally:
    print("\nFim.")

Sanvando ... resumo_gasolina.csv
Sanvando ... resumo_etanol.csv
Sanvando ... resumo_diesel.csv

Fim.


In [14]:
import pandas as pd

resumo_gasolina = rf"C:\Users\Nino\AnacondaProjects\combustivel\resumo_gasolina.csv"
df = pd.read_csv(
    resumo_gasolina,
    encoding='latin1',
    sep=',',
    usecols=['ESTADO', 'PRODUTO', 'MES_ANO', 'VL_VENDA']  # Especifica apenas as colunas que quer manter
)
anos = ['2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022','2023','2024']
mes = '04'
for ano in anos:
    df_filtrado = df[df['MES_ANO'] == f'{mes}/{ano}']
    print(f"Ano {ano} - GASOLINA - {len(df_filtrado)}")

print(df.head())


Ano 2004 - GASOLINA - 27
Ano 2005 - GASOLINA - 27
Ano 2006 - GASOLINA - 27
Ano 2007 - GASOLINA - 27
Ano 2008 - GASOLINA - 27
Ano 2009 - GASOLINA - 27
Ano 2010 - GASOLINA - 27
Ano 2011 - GASOLINA - 27
Ano 2012 - GASOLINA - 27
Ano 2013 - GASOLINA - 27
Ano 2014 - GASOLINA - 26
Ano 2015 - GASOLINA - 27
Ano 2016 - GASOLINA - 27
Ano 2017 - GASOLINA - 27
Ano 2018 - GASOLINA - 27
Ano 2019 - GASOLINA - 27
Ano 2020 - GASOLINA - 27
Ano 2021 - GASOLINA - 27
Ano 2022 - GASOLINA - 27
Ano 2023 - GASOLINA - 27
Ano 2024 - GASOLINA - 27
  ESTADO   PRODUTO  MES_ANO  VL_VENDA
0     AC  GASOLINA  01/2004  2.000000
1     AC  GASOLINA  01/2005  2.025641
2     AC  GASOLINA  01/2006  2.216495
3     AC  GASOLINA  01/2007  2.542857
4     AC  GASOLINA  01/2008  2.513514


In [15]:
import pandas as pd

resumo_etanol = rf"C:\Users\Nino\AnacondaProjects\combustivel\resumo_etanol.csv"
df = pd.read_csv(
    resumo_etanol,
    encoding='latin1',
    sep=',',
    usecols=['ESTADO', 'PRODUTO', 'MES_ANO', 'VL_VENDA']  # Especifica apenas as colunas que quer manter
)
anos = ['2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022','2023','2024']
mes = '04'
for ano in anos:
    df_filtrado = df[df['MES_ANO'] == f'{mes}/{ano}']
    print(f"Ano {ano} - ETANOL - {len(df_filtrado)}")

print(df.head())


Ano 2004 - ETANOL - 27
Ano 2005 - ETANOL - 27
Ano 2006 - ETANOL - 27
Ano 2007 - ETANOL - 27
Ano 2008 - ETANOL - 27
Ano 2009 - ETANOL - 27
Ano 2010 - ETANOL - 27
Ano 2011 - ETANOL - 27
Ano 2012 - ETANOL - 27
Ano 2013 - ETANOL - 27
Ano 2014 - ETANOL - 26
Ano 2015 - ETANOL - 27
Ano 2016 - ETANOL - 26
Ano 2017 - ETANOL - 27
Ano 2018 - ETANOL - 27
Ano 2019 - ETANOL - 26
Ano 2020 - ETANOL - 27
Ano 2021 - ETANOL - 27
Ano 2022 - ETANOL - 26
Ano 2023 - ETANOL - 27
Ano 2024 - ETANOL - 27
  ESTADO PRODUTO  MES_ANO  VL_VENDA
0     AC  ETANOL  01/2004  1.107692
1     AC  ETANOL  01/2005  1.222222
2     AC  ETANOL  01/2006  2.012821
3     AC  ETANOL  01/2007  1.333333
4     AC  ETANOL  01/2008  1.989474


In [16]:
import pandas as pd

resumo_diesel = rf"C:\Users\Nino\AnacondaProjects\combustivel\resumo_diesel.csv"
df = pd.read_csv(
    resumo_diesel,
    encoding='latin1',
    sep=',',
    usecols=['ESTADO', 'PRODUTO', 'MES_ANO', 'VL_VENDA']  # Especifica apenas as colunas que quer manter
)
anos = ['2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022','2023','2024']
mes = '04'
for ano in anos:
    df_filtrado = df[df['MES_ANO'] == f'{mes}/{ano}']
    print(f"Ano {ano} - DIESEL - {len(df_filtrado)}")

print(df.head())

Ano 2004 - DIESEL - 27
Ano 2005 - DIESEL - 27
Ano 2006 - DIESEL - 27
Ano 2007 - DIESEL - 27
Ano 2008 - DIESEL - 27
Ano 2009 - DIESEL - 27
Ano 2010 - DIESEL - 27
Ano 2011 - DIESEL - 27
Ano 2012 - DIESEL - 27
Ano 2013 - DIESEL - 27
Ano 2014 - DIESEL - 26
Ano 2015 - DIESEL - 27
Ano 2016 - DIESEL - 27
Ano 2017 - DIESEL - 27
Ano 2018 - DIESEL - 27
Ano 2019 - DIESEL - 27
Ano 2020 - DIESEL - 27
Ano 2021 - DIESEL - 27
Ano 2022 - DIESEL - 27
Ano 2023 - DIESEL - 27
Ano 2024 - DIESEL - 27
  ESTADO PRODUTO  MES_ANO  VL_VENDA
0     AC  DIESEL  01/2004  1.139535
1     AC  DIESEL  01/2005  1.582609
2     AC  DIESEL  01/2006  2.000000
3     AC  DIESEL  01/2007  2.000000
4     AC  DIESEL  01/2008  2.000000


# Gerar HTML a partir de resumos csv

In [49]:
import pandas as pd
import json
import os
from pathlib import Path

labels = []
for ano in range(2003, 2026):
    for mes in range(1, 13):
        labels.append(f"{mes:02d}/{ano}")

base_path = Path(rf"C:\Users\Nino\AnacondaProjects\combustivel")
arquivos = {
    'GASOLINA': base_path / 'resumo_gasolina.csv',
    'ETANOL': base_path / 'resumo_etanol.csv',
    'DIESEL': base_path / 'resumo_diesel.csv',
    'EVENTS': base_path / 'eventos.csv'
}

# Inicializar a estrutura de dados
fuel_data = {
    'GASOLINA': {'labels': labels, 'datasets': []}, 
    'ETANOL': {'labels': labels, 'datasets': []},
    'DIESEL': {'labels': labels, 'datasets': []},
    'EVENTS': []
}

# Cores para os estados (pode personalizar conforme necessário)
cores_estados = {
    'AC': 'rgba(0,255,0,1)',
    'AL': 'rgba(0,100,0,1)',
    'AM': 'rgba(218,165,32,1)',
    'AP': 'rgba(139,69,19,1)',
    'BA': 'rgba(188,143,143,1)',
    'CE': 'rgba(210,105,30,1)',
    'DF': 'rgba(135,206,235,1)',
    'ES': 'rgba(75,0,130,1)',
    'GO': 'rgba(139,0,139,1)',
    'MA': 'rgba(255,20,147,1)',
    'MG': 'rgba(255,192,203,1)',
    'MS': 'rgba(220,20,60,1)',
    'MT': 'rgba(255,215,0,1)',
    'PA': 'rgba(255,255,0,1)',
    'PB': 'rgba(176,224,230,1)',
    'PE': 'rgba(255,165,0,1)',
    'PI': 'rgba(255,105,180,1)',
    'PR': 'rgba(221,160,221,1)',
    'RJ': 'rgba(222,184,135,1)',
    'RN': 'rgba(189,83,107,1)',
    'RO': 'rgba(60,179,113,1)',
    'RR': 'rgba(143,188,143,1)',
    'RS': 'rgba(95,158,160,1)',
    'SC': 'rgba(0,255,255,1)',
    'SE': 'rgba(70,130,180,1)',
    'SP': 'rgba(30,144,255,1)',
    'TO': 'rgba(131,111,255,1)'
}

def processar_produtos():
    # Processar os arquivos de combustíveis
    for produto in ['GASOLINA', 'ETANOL', 'DIESEL']:
        try:
            arquivo = arquivos[produto]
            try:
                df = pd.read_csv(arquivo, encoding='utf-8')
            except UnicodeDecodeError:
                df = pd.read_csv(arquivo, encoding='latin1')
            
            # Garantir que os dados estejam na ordem correta dos labels
            df['ORDEM'] = df['MES_ANO'].apply(lambda x: labels.index(x) if x in labels else len(labels))
            df = df.sort_values('ORDEM')
            df = df.drop('ORDEM', axis=1)
            
            # Adicionar datasets para cada estado
            for estado, grupo in df.groupby('ESTADO'):
                # Criar lista de valores na ordem correta dos labels
                dados_estado = []
                for mes_ano in labels:
                    valor = grupo[grupo['MES_ANO'] == mes_ano]['VL_VENDA']
                    dados_estado.append(valor.mean() if not valor.empty else None)
                
                dataset = {
                    'uf': estado,
                    'label': f"{estado} - {produto.capitalize()}",
                    'data': dados_estado,
                    'borderColor': cores_estados.get(estado, 'rgba(0,0,0,1)'),
                    'backgroundColor': cores_estados.get(estado, 'rgba(0,0,0,1)').replace('1)', '0.1)'),
                    'hidden': False
                }
                
                fuel_data[produto]['datasets'].append(dataset)
            
            print(f"Processado {produto} com {len(df)} registros")
            
        except Exception as e:
            print(f"Erro ao processar {produto}: {str(e)}")

def processar_eventos():
    # Processar os eventos (formato já compatível MM/YYYY)
    try:
        try:
            eventos_df = pd.read_csv(arquivos['EVENTS'], encoding='utf-8')
        except UnicodeDecodeError:
            eventos_df = pd.read_csv(arquivos['EVENTS'], encoding='latin1')
        
        # Verificar e padronizar o formato da data
        eventos_df['MES_ANO'] = eventos_df['date'].str.strip()  # Remove espaços extras
        
        # Verificar quais eventos têm datas correspondentes nos labels
        eventos_validos = []
        for _, row in eventos_df.iterrows():
            if row['MES_ANO'] in labels:
                evento = {
                    'date': row['MES_ANO'],
                    'title': row['title'],
                    'description': row['description'],
                    'impact': row['impact']
                }
                eventos_validos.append(evento)
                print(f"Evento adicionado: {evento['title']} em {evento['date']}")
            else:
                print(f"Evento ignorado (data não encontrada): {row['title']} em {row['MES_ANO']}")
        
        fuel_data['EVENTS'] = eventos_validos
        print(f"Processados {len(eventos_validos)} eventos válidos")
        
    except Exception as e:
        print(f"Erro ao processar eventos: {str(e)}")

def gerar_json():
    # Salvar os dados em um arquivo JSON com tratamento especial de codificação
    output_path = base_path / 'fuelData.json'
    
    try:
        # Primeira tentativa com UTF-8
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(fuel_data, f, ensure_ascii=False, indent=2)
    except UnicodeEncodeError:
        # Fallback para latin1 se necessário
        with open(output_path, 'w', encoding='latin1') as f:
            json.dump(fuel_data, f, ensure_ascii=False, indent=2)
    
    # Verificação pós-escrita
    try:
        with open(output_path, 'r', encoding='utf-8') as f:
            content = f.read()
        print(f"JSON gerado com sucesso em UTF-8: {output_path}")
    except UnicodeDecodeError:
        print("Aviso: O arquivo JSON não está em UTF-8 válido")
        # Forçar conversão para UTF-8
        with open(output_path, 'r', encoding='latin1') as f:
            content = f.read()
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(content)
        print("Arquivo convertido para UTF-8")

def processar_arquivos_csv():    
    processar_produtos()
    processar_eventos()
    gerar_json()
    gerar_html_completo(base_path, fuel_data)
    
    return fuel_data

def criar_template_basico(template_path):

    estilo = """
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 20px;
            background-color: #f5f5f5;
        }
        .container {
            max-width: 1200px;
            margin: 0 auto;
            background-color: white;
            padding: 20px;
            border-radius: 8px;
            box-shadow: 0 0 10px #000;
        }
        .chart-container {
            height: 500px;
            margin: 20px 0;
            position: relative;
        }
        .controls {
            margin-bottom: 20px;
            padding: 15px;
            background-color: #f8f9fa;
            border-radius: 5px;
            position: relative;
        }
        select{
            width: 12px;
            height: 200px;
        } 
        button {
            padding: 8px 12px;
            border-radius: 4px;
            margin-right: 10px;
        }
        button {
            background-color: #4CAF50;
            color: white;
            border: none;
            cursor: pointer;
        }
        .uf {
            display: flex;
            flex: left;
            gap: 10px;
            margin-top: 15px;
            position: relative;
            padding-bottom: 50px;
            min-height: 150px;
            width:auto;
        }
        .legend {
            display: flex;
            flex-wrap: wrap;
            gap: 8px;
            padding: 10px;
            background-color: #f8f9fa;
            border-radius: 5px;
            border: 1px solid #ddd;
            margin-top: 15px;
        }
        .legend-item {
            display: flex;
            align-items: center;
            padding: 4px 8px;
            background-color: white;
            border-radius: 4px;
            border: 1px solid #eee;
            box-shadow: 0 1px 2px #000;
            white-space: nowrap;
        }
        .legend-color {
            width: 15px;
            height: 15px;
            margin-right: 5px;
            border: 1px solid #ddd;
        }
        .product-tabs {
            display: flex;
            margin-bottom: 15px;
        }
        .product-tab {
            padding: 8px 16px;
            background-color: #e0e0e0;
            margin-right: 5px;
            cursor: pointer;
            border-radius: 4px 4px 0 0;
        }
        .product-tab.active {
            background-color: #4CAF50;
            color: white;
        }
        #ufFilter {
            width: 50px;
            min-height: 100px;
        }
        .event-tooltip {
            position: absolute;
            padding: 12px;
            background: rgba(0, 0, 0, 0.9);
            color: white;
            border-radius: 6px;
            pointer-events: none;
            z-index: 1000;
            max-width: 280px;
            font-size: 14px;
            box-shadow: 0 3px 10px rgba(0,0,0,0.3);
            border-left: 3px solid;
            transition: opacity 0.2s;
        }

        .event-tooltip::after {
            content: '';
            position: absolute;
            left: 50%;
            bottom: -10px;
            transform: translateX(-50%);
            border-width: 5px;
            border-style: solid;
            border-color: rgba(0, 0, 0, 0.9) transparent transparent transparent;
        }
        .event-tooltip strong {
            color: #ffcc00;
        }
        .event-tooltip em {
            color: #730303;
            font-size: 22px;
        }
        .event-card {
            margin: 10px 0;
            padding: 15px;
            border-left: 4px solid #9966ff;
            background-color: #f9f9f9;
            border-radius: 4px;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            transition: transform 0.2s;
        }

        .event-card:hover {
            transform: translateY(-2px);
            box-shadow: 0 4px 8px rgba(0,0,0,0.15);
        }

        .event-card .event-date {
            font-weight: bold;
            color: #666;
            margin-bottom: 5px;
        }

        .event-card .event-title {
            font-size: 18px;
            font-weight: bold;
            margin-bottom: 8px;
        }

        .event-card .event-description {
            color: #333;
            margin-bottom: 8px;
        }

        .event-card .event-impact {
            font-size: 14px;
            color: #555;
        }
        .grupo {
            top: 15px;
            padding: 8px 16px;
            right: 15px;
            display: flex;
            background-color: #edebab;
            margin:15px;
            
        }
        .zoom-buttons {
            position: absolute;
            top: 15px;
            right: 15px;
            display: flex;
            gap: 5px;
        }
        .zoom-button {
            background-color: #4CAF50;
            color: white;
            border: none;
            border-radius: 4px;
            width: 50px;
            height: 50px;
            display: flex;
            align-items: center;
            justify-content: center;
            cursor: pointer;
            font-weight: bold;
            font-size: 16px;
            padding:15px;
        }
        .zoom-button:hover {
            background-color: #45a049;
        }
        .nav-buttons {
            position: absolute;
            top: 60px;
            right: 60px;
            display: flex;
            gap: 5px;
            padding:15px;
        }
        .nav-button {
            background-color: #2196F3;
            color: white;
            border: none;
            border-radius: 4px;
            width: 50px;
            height: 50px;
            display: flex;
            align-items: center;
            justify-content: center;
            cursor: pointer;
            font-weight: bold;
            font-size: 16px;
        }
        .nav-button:hover {
            background-color: #0b7dda;
        }
        .filter-section {
            margin-top: 50px;
        }
        #applyFilter{
            height:50px;
            padding:15px;
            margin:15px;
        }
        .grupo {
            top: 15px;
            padding: 8px 16px;
            right: 15px;
            display: flex;
            background-color: #edebab;
            margin:15px;
            
        }
    """
    conteudo = """ 
    <div class="container">
        <h1>Histórico de preço médio de combustível por Estado</h1>

        <div class="product-tabs" id="productTabs">
            <div class="product-tab active" data-product="GASOLINA">Gasolina</div>
            <div class="product-tab" data-product="ETANOL">Etanol</div>
            <div class="product-tab" data-product="DIESEL">Diesel</div>
        </div>

        <div class="controls">
            
            
            <div class="filter-section">
                <div class="uf">
                    <label for="ufFilter">Filtrar UFs:</label>
                    <select id="ufFilter" multiple>
                        <!-- As opções serão preenchidas pelo JavaScript -->
                    </select>
                    <button id="applyFilter" style="width: 100px; height: 50px; font-size: 14px;">Aplicar Filtro</button>
                </div>
                <div class="legend" id="chartLegend"></div>
            </div>

            <div class="grupo">
                <button class="nav-button" id="navLeft" onclick="navigateLeft()">←</button>
                <button class="nav-button" id="navRight" onclick="navigateRight()">→</button>
            </div>
            
            <div class="grupo">
                <button class="zoom-button" id="zoomIn">+</button>
                <button class="zoom-button" id="zoomOut">-</button>
                <button class="zoom-button" id="resetZoom">⟲</button>
            </div>
        </div>

        <div class="chart-container">
            <canvas id="priceChart"></canvas>
        </div>

        <div class="events-panel">
            <div id="eventsContainer">
                <!-- Os eventos serão inseridos aqui pelo JavaScript -->
            </div>
        </div>
    </div>
    """
    javascript = """ 
    // Dados serão injetados aqui
    const fuelData = {};

    // Variáveis globais
    let priceChart;
    let currentProduct = 'GASOLINA';
    let visibleRange = 6;

    // Inicializar controles
    function initializeControls() {
        // Inicializar tabs de produtos
        document.querySelectorAll('.product-tab').forEach(tab => {
            tab.addEventListener('click', function() {
                document.querySelectorAll('.product-tab').forEach(t => t.classList.remove('active'));
                this.classList.add('active');
                currentProduct = this.dataset.product;
                updateChart();
            });
        });

        // Inicializar seletor de UFs
        const ufFilter = document.getElementById('ufFilter');
        if (ufFilter.children.length === 0) {
            const ufs = [...new Set([
                ...fuelData.GASOLINA.datasets.map(d => d.uf),
                ...fuelData.ETANOL.datasets.map(d => d.uf),
                ...fuelData.DIESEL.datasets.map(d => d.uf)
            ])].sort();
            
            ufs.forEach(uf => {
                const option = document.createElement('option');
                option.value = uf;
                option.textContent = uf;
                option.selected = true;
                ufFilter.appendChild(option);
            });
        }

        // Configurar botão de filtro
        document.getElementById('applyFilter').addEventListener('click', updateChart);
        
        // Configurar botões de zoom
        document.getElementById('zoomIn').addEventListener('click', zoomIn);
        document.getElementById('zoomOut').addEventListener('click', zoomOut);
        document.getElementById('resetZoom').addEventListener('click', resetZoom);
        
        // Configurar botões de navegação
        document.getElementById('navLeft').addEventListener('click', navigateLeft);
        document.getElementById('navRight').addEventListener('click', navigateRight);
    }

    // Funções de zoom
    function zoomIn() {
        if (!priceChart) return;
        
        const xAxis = priceChart.options.scales.x;
        if (!xAxis) return;
        
        if (xAxis.min === undefined || xAxis.max === undefined) {
            xAxis.min = 0;
            xAxis.max = Math.min(visibleRange, priceChart.data.labels.length - 1);
        }
        
        const range = xAxis.max - xAxis.min;
        const newRange = Math.max(2, range * 0.7);
        
        const center = xAxis.min + range / 2;
        xAxis.min = Math.max(0, center - newRange / 2);
        xAxis.max = Math.min(priceChart.data.labels.length - 1, center + newRange / 2);
        
        priceChart.update();
    }

    function zoomOut() {
        if (!priceChart) return;
        
        const xAxis = priceChart.options.scales.x;
        if (!xAxis) return;
        
        if (xAxis.min === undefined || xAxis.max === undefined) return;
        
        const range = xAxis.max - xAxis.min;
        const newRange = range * 1.3;
        const center = xAxis.min + range / 2;
        
        xAxis.min = Math.max(0, center - newRange / 2);
        xAxis.max = Math.min(priceChart.data.labels.length - 1, center + newRange / 2);
        
        if (xAxis.max - xAxis.min >= priceChart.data.labels.length) {
            resetZoom();
            return;
        }
        
        priceChart.update();
    }

    // Reset Zoom - Mostrar todo o período
    function resetZoom() {
        if (!priceChart) return;
        
        const xAxis = priceChart.options.scales.x;
        if (!xAxis) return;
        
        // Resetar para mostrar todo o período
        xAxis.min = undefined;
        xAxis.max = undefined;
        priceChart.update();
    }

    // Funções de navegação
    function navigateLeft() {
        if (!priceChart) return;
        
        const xAxis = priceChart.options.scales.x;
        if (!xAxis) return;
        
        // Se não há zoom aplicado, não faz nada
        if (xAxis.min === undefined || xAxis.max === undefined) {
            return;
        }
        
        const range = xAxis.max - xAxis.min;
        const step = Math.max(1, Math.floor(range * 0.3)); // Ajuste o passo aqui (30% do range)
        
        // Calcula os novos limites
        let newMin = xAxis.min - step;
        let newMax = xAxis.max - step;
        
        // Verifica os limites do gráfico
        if (newMin < 0) {
            newMin = 0;
            newMax = range;
        }
        
        // Aplica os novos valores
        xAxis.min = newMin;
        xAxis.max = newMax;
        
        priceChart.update();
    }

    function navigateRight() {
        if (!priceChart) return;
        
        const xAxis = priceChart.options.scales.x;
        if (!xAxis) return;
        
        // Se não há zoom aplicado, não faz nada
        if (xAxis.min === undefined || xAxis.max === undefined) {
            return;
        }
        
        const range = xAxis.max - xAxis.min;
        const step = Math.max(1, Math.floor(range * 0.3)); // Ajuste o passo aqui (30% do range)
        const maxIndex = priceChart.data.labels.length - 1;
        
        // Calcula os novos limites
        let newMin = xAxis.min + step;
        let newMax = xAxis.max + step;
        
        // Verifica os limites do gráfico
        if (newMax > maxIndex) {
            newMax = maxIndex;
            newMin = maxIndex - range;
        }
        
        // Aplica os novos valores
        xAxis.min = newMin;
        xAxis.max = newMax;
        
        priceChart.update();
    }

    function updateChart() {
        const selectedUFs = Array.from(document.getElementById('ufFilter').selectedOptions)
            .map(option => option.value);
        
        const currentData = fuelData[currentProduct];
        const filteredDatasets = currentData.datasets.filter(d => selectedUFs.includes(d.uf));
        
        if (priceChart) {
            priceChart.destroy();
        }
        
        const ctx = document.getElementById('priceChart').getContext('2d');
        
        // Configurar anotações para os eventos
        const eventAnnotations = fuelData.EVENTS.map(event => {
            const eventIndex = currentData.labels.indexOf(event.date);
            if (eventIndex === -1) return null;

            return {
                type: 'line',
                mode: 'vertical',
                scaleID: 'x',
                value: eventIndex,
                borderColor: getImpactColor(event.impact),
                borderWidth: 2,
                borderDash: [5, 5],
                label: {
                    content: event.title,
                    enabled: true,
                    position: 'top',
                    backgroundColor: 'rgba(0,0,0,0.7)',
                    color: '#fff',
                    font: {
                        size: 10
                    },
                    rotation: 0,
                    xAdjust: 0,
                    yAdjust: -20
                }
            };
        }).filter(annotation => annotation !== null);
        
        // Função auxiliar para cores baseadas no impacto
        function getImpactColor(impact) {
            const colors = {
                'alto': 'rgba(255, 99, 132, 0.7)',
                'médio': 'rgba(255, 159, 64, 0.7)',
                'baixo': 'rgba(75, 192, 192, 0.7)'
            };
            return colors[impact.toLowerCase()] || 'rgba(153, 102, 255, 0.7)';
        }
        
        priceChart = new Chart(ctx, {
            type: 'line',
            data: {
                labels: currentData.labels,
                datasets: filteredDatasets.map(dataset => ({
                    label: dataset.label,
                    data: dataset.data,
                    borderColor: dataset.borderColor,
                    backgroundColor: dataset.backgroundColor,
                    borderWidth: 2,
                    tension: 0.1
                }))
            },
            options: {
                responsive: true,
                maintainAspectRatio: false,
                scales: {
                    y: {
                        beginAtZero: false,
                        title: {
                            display: true,
                            text: 'Preço (R$/litro)'
                        }
                    },
                    x: {
                        title: {
                            display: true,
                            text: 'Período'
                        },
                        ticks: {
                            autoSkip: true,
                            maxRotation: 45,
                            minRotation: 45
                        }
                    }
                },
                plugins: {
                    annotation: {
                        annotations: eventAnnotations
                    },
                    legend: {
                        display: false
                    }
                }
            }
        });
        
        updateLegend(filteredDatasets);
        addEventTooltips();
    }

    // Atualizar legenda personalizada
    function updateLegend(datasets) {
        const legendContainer = document.getElementById('chartLegend');
        legendContainer.innerHTML = '';
        
        datasets.forEach((dataset, index) => {
            const legendItem = document.createElement('div');
            legendItem.className = 'legend-item';
            
            const colorBox = document.createElement('div');
            colorBox.className = 'legend-color';
            colorBox.style.backgroundColor = dataset.borderColor;
            
            const label = document.createElement('span');
            label.textContent = dataset.label;
            
            legendItem.appendChild(colorBox);
            legendItem.appendChild(label);
            
            legendItem.addEventListener('click', () => {
                const meta = priceChart.getDatasetMeta(index);
                meta.hidden = !meta.hidden;
                priceChart.update();
                legendItem.style.opacity = meta.hidden ? '0.5' : '1';
            });
            
            legendContainer.appendChild(legendItem);
        });
    }

    function addEventTooltips() {
        const chartCanvas = document.getElementById('priceChart');
        const eventTooltip = document.createElement('div');
        eventTooltip.className = 'event-tooltip';
        eventTooltip.style.position = 'absolute';
        eventTooltip.style.padding = '8px';
        eventTooltip.style.background = 'rgba(0, 0, 0, 0.8)';
        eventTooltip.style.color = 'white';
        eventTooltip.style.borderRadius = '4px';
        eventTooltip.style.pointerEvents = 'none';
        eventTooltip.style.display = 'none';
        eventTooltip.style.zIndex = '100';
        chartCanvas.parentNode.appendChild(eventTooltip);

        chartCanvas.addEventListener('mousemove', function(e) {
            const rect = chartCanvas.getBoundingClientRect();
            const x = e.clientX - rect.left;
            const y = e.clientY - rect.top;
            
            const activeEvent = fuelData.EVENTS.find(event => {
                const eventIndex = fuelData[currentProduct].labels.indexOf(event.date);
                if (eventIndex === -1) return false;
                
                const point = priceChart.scales.x.getPixelForValue(eventIndex);
                return Math.abs(x - point) < 10;
            });
            
            if (activeEvent) {
                eventTooltip.style.display = 'block';
                eventTooltip.style.left = `${e.clientX + 10}px`;
                eventTooltip.style.top = `${e.clientY + 10}px`;
                eventTooltip.innerHTML = `
                <div style="font-size: 20px;">${activeEvent.title}</div>
                <div style="color: #aaa; margin-bottom: 8px;">${activeEvent.date}</div>
                <div style="margin-bottom: 8px;">${activeEvent.description}</div>
                <em style="color: ${getImpactColor(activeEvent.impact)};">Impacto: ${activeEvent.impact}</em>
            `;
            } else {
                eventTooltip.style.display = 'none';
            }
        });
        
        chartCanvas.addEventListener('mouseout', function() {
            eventTooltip.style.display = 'none';
        });
    }

    function displayEvents() {
        const eventsContainer = document.getElementById('eventsContainer');
        eventsContainer.innerHTML = '<h2>Eventos Relevantes</h2>';

        if (!fuelData.EVENTS || fuelData.EVENTS.length === 0) {
            eventsContainer.innerHTML += '<p>Nenhum evento encontrado.</p>';
            return;
        }

        // Ordenar eventos do mais recente para o mais antigo
        const sortedEvents = [...fuelData.EVENTS].sort((a, b) => {
            const [aMonth, aYear] = a.date.split('/').map(Number);
            const [bMonth, bYear] = b.date.split('/').map(Number);
            return new Date(bYear, bMonth - 1) - new Date(aYear, aMonth - 1);
        });

        sortedEvents.forEach(event => {
            const impactColor = getImpactColor(event.impact);
            
            const eventCard = document.createElement('div');
            eventCard.className = 'event-card';
            eventCard.style.cssText = `
                margin: 10px 0px;
                padding: 15px;
                border-left: 4px solid ${impactColor};
                background-color: #f9f9f9;
                border-radius: 4px;
                box-shadow: rgba(0, 0, 0, 0.1) 0px 2px 4px;
            `;
            
            eventCard.innerHTML = `
                <div style="display: flex; justify-content: space-between; margin-bottom: 8px;">
                    <div style="font-weight: bold; color: ${impactColor};">
                        ${event.title}
                    </div>
                    <div style="color: #666;">
                        ${event.date}
                    </div>
                </div>
                <div style="margin-bottom: 8px; color: #333;">
                    ${event.description}
                </div>
                <div style="font-size: 14px; color: #555;">
                    <strong>Impacto:</strong> ${event.impact}
                </div>
            `;
            
            eventsContainer.appendChild(eventCard);
        });
    }

    function getImpactColor(impact) {
        impact = (impact || '').toLowerCase();
        if (impact.includes('alto')) return '#ff6384';
        if (impact.includes('médio') || impact.includes('medio')) return '#ff9f40';
        if (impact.includes('baixo')) return '#4bc0c0';
        return '#9966ff';
    }

    // Função auxiliar para determinar a cor com base no impacto
    function getImpactColor(impact) {
        impact = (impact || '').toLowerCase();
        if (impact.includes('alto')) return '#ff6384';
        if (impact.includes('médio') || impact.includes('medio')) return '#ff9f40';
        if (impact.includes('baixo')) return '#4bc0c0';
        return '#9966ff';
    }

    // Inicializar a página
    window.onload = function() {
        initializeControls();
        updateChart();
        displayEvents();
    };
"""
    template = f"""<!DOCTYPE html>
        <html lang="pt-BR">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>Preços de Combustíveis por UF</title>
            <script src="https://cdn.jsdelivr.net/npm/chart.js@3.7.1/dist/chart.min.js"></script>
            <script src="https://cdn.jsdelivr.net/npm/chartjs-plugin-annotation@1.3.0/dist/chartjs-plugin-annotation.min.js"></script>
            <style>
                {estilo}
            </style>
        </head>
        <body>
            {conteudo}
            <script>
                {javascript}
            </script>
        </body>
        </html>
        """
    
    with open(template_path, 'w', encoding='utf-8') as f:
        f.write(template)

def gerar_html_completo(base_path, fuel_data):
    # Ler o template HTML
    template_path = base_path / 'template.html'
    output_html_path = base_path / 'precos_combustiveis.html'
    
    # Se não existir um template, criar um básico
    if not template_path.exists():
        criar_template_basico(template_path)
    
    # Ler o template
    with open(template_path, 'r', encoding='utf-8') as f:
        html_content = f.read()
    
    # Substituir os dados no template
    html_content = html_content.replace(
        'const fuelData = {};',
        f'const fuelData = {json.dumps(fuel_data, ensure_ascii=False, indent=4)};'
    )
    
    # Salvar o HTML completo
    with open(output_html_path, 'w', encoding='utf-8') as f:
        f.write(html_content)
    
    print(f"HTML completo gerado em {output_html_path}")

if __name__ == "__main__":
    dados = processar_arquivos_csv()

Processado GASOLINA com 6820 registros
Processado ETANOL com 6786 registros
Processado DIESEL com 6818 registros
Evento adicionado: E1 em 03/2003
Evento adicionado: E2 em 01/2019
Evento adicionado: E3 em 03/2020
Evento adicionado: E4 em 02/2022
Evento adicionado: E5 em 03/2025
Processados 5 eventos válidos
JSON gerado com sucesso em UTF-8: C:\Users\Nino\AnacondaProjects\combustivel\fuelData.json
HTML completo gerado em C:\Users\Nino\AnacondaProjects\combustivel\precos_combustiveis.html


# Se executou tudo você deve ter gerado o arquivo preco_combustível.html